In [1]:
import os
import sys
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import pandas as pd
import torchaudio
import codes.utils
from codes.preprocess_data import *

In [2]:
main = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [3]:
FOLDER_ANNOTATIONS = "datos/"
ANNOTATIONS_FILE = ["train_set.txt","test_set.txt"]
N_FOLDS = 4
MBE_DIR_TR = "features/"+ ANNOTATIONS_FILE[0].split(".")[0]+"/"
MBE_DIR_TE = "features/"+ ANNOTATIONS_FILE[1].split(".")[0]+"/"
SAMPLE_RATE = 44100
LEN_SEC = 300
LEN_SAMPLES = LEN_SEC*SAMPLE_RATE
__class_labels = {
'perro'    : 0,
'rana'     : 1,
'lluvia'   : 2,
'motor'    : 3,
'ave'      : 4
}
N_FFT = 2048
HOP = int(N_FFT/2)
N_MELS = 40

In [4]:
REF = "ref_num_file.txt"
df_wet = pd.read_csv(FOLDER_ANNOTATIONS + REF, sep = " ")
display(df_wet)

,humedal,fecha,first_file,last_file
0,Angachilla,20200109,1,24
1,Angachilla,20191027,169,192
2,Angachilla,20191028,289,312
3,Angachilla,20191029,313,336
4,Angachilla,20191030,337,360
...,...,...,...,...
70,El_Bosque,20201027,1681,1704
71,El_Bosque,20201028,1705,1728
72,El_Bosque,20201029,1729,1752
73,El_Bosque,20201030,1753,1776


In [5]:
wets = {"Angachilla":[],"El_Bosque":[],"Miraflores":[]}

In [6]:
df_train = pd.read_csv(FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[0],sep=" ",
                    names=["filepath","start","end","class_name"])
audio_filenames_train = np.unique(df_train.filepath)

In [7]:
df_test = pd.read_csv(FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[1],sep=" ",
                    names=["filepath","start","end","class_name"])
audio_filenames_test = np.unique(df_test.filepath)

In [8]:
for i,file in enumerate(audio_filenames_train):
    mbe,label = preprocess_data(file,main,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[0],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    k = codes.utils.num_2wet(df_wet,int(numberfile))
    wets[k].append(mbe)
    #namembe = "{}mbe{}.pt".format(MBE_DIR_TR,numberfile)
    #namembe = os.path.join(main,namembe)
    #torch.save({"data":mbe,"label":label}, namembe)
    #mean_std = torch.cat((mean_std,mbe))

/home/vpoblete/Escritorio/tesis/notebooks/sednet_pytorch/codes/preprocess_data.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensor = torch.tensor(label,device=device)


In [9]:
for i,file in enumerate(audio_filenames_test):
    mbe,label = preprocess_data(file,main,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[1],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    k = codes.utils.num_2wet(df_wet,int(numberfile))
    wets[k].append(mbe)
    #namembe = "{}mbe{}.pt".format(MBE_DIR_TE,numberfile)
    #namembe = os.path.join(main,namembe)
    #torch.save({"data":mbe,"label":label}, namembe)
    #mean_std = torch.cat((mean_std,mbe))

In [10]:
mean = dict()
std = dict()
for k in wets.keys():
    mean[k] = torch.mean(torch.stack(wets[k]))
    std[k] = torch.std(torch.stack(wets[k]))
print(mean, std)

{'Angachilla': tensor(-1.0951, device='cuda:0'), 'El_Bosque': tensor(-0.4757, device='cuda:0'), 'Miraflores': tensor(-1.7114, device='cuda:0')} {'Angachilla': tensor(2.6212, device='cuda:0'), 'El_Bosque': tensor(2.4989, device='cuda:0'), 'Miraflores': tensor(2.8110, device='cuda:0')}


In [11]:
wets_z = {"Angachilla":[],"El_Bosque":[],"Miraflores":[]}

In [12]:
for i,file in enumerate(audio_filenames_train):
    mbe,label = preprocess_data(file,main,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[0],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    namembe = "{}mbe{}.pt".format(MBE_DIR_TR,numberfile)
    k = codes.utils.num_2wet(df_wet,int(numberfile))
    namembe = os.path.join(main,namembe)
    mbe = (mbe-mean[k])/std[k]
    torch.save({"data":mbe,"label":label}, namembe)
    wets_z[k].append(mbe)

In [13]:
for i,file in enumerate(audio_filenames_test):

    mbe,label = preprocess_data(file,main,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[1],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    namembe = "{}mbe{}.pt".format(MBE_DIR_TE,numberfile)
    k = codes.utils.num_2wet(df_wet,int(numberfile))
    namembe = os.path.join(main,namembe)
    mbe = (mbe-mean[k])/std[k]
    torch.save({"data":mbe,"label":label}, namembe)
    wets_z[k].append(mbe)

In [14]:
mean_z = dict()
std_z = dict()
for k in wets.keys():
    mean_z[k] = torch.mean(torch.stack(wets_z[k]))
    std_z[k] = torch.std(torch.stack(wets_z[k]))
print(mean_z, std_z)

{'Angachilla': tensor(-1.8613e-08, device='cuda:0'), 'El_Bosque': tensor(-1.4235e-08, device='cuda:0'), 'Miraflores': tensor(1.6755e-08, device='cuda:0')} {'Angachilla': tensor(1.0000, device='cuda:0'), 'El_Bosque': tensor(1., device='cuda:0'), 'Miraflores': tensor(1.0000, device='cuda:0')}
